In [34]:
import pandas as pd
import numpy as np
import haversine as hs
#!pip install haversine


pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
pd.set_option('display.precision', 0)
pd.options.display.float_format = "{:,.0f}".format

In [35]:
data = pd.read_csv("./data/kc_house_data.csv")
df_fortune_500s = pd.read_excel("./data/Fortune500LatsLongs.xlsx")

In [36]:
#Merging Cross Ref Df with Points of Intereste Df
# CPInts_With_Rank = CPInts.merge(CPInts_Xref, on="CODE", how="left", indicator=True)

# CPInts_With_Rank['tup_lat_Long'] = list(zip(CPInts_With_Rank.Y, CPInts_With_Rank.X, CPInts_With_Rank.OBJECTID, CPInts_With_Rank.NAME,CPInts_With_Rank.PointofInterest_Rank))
df_fortune_500s['tup_lat_Long_500'] = list(zip(df_fortune_500s.Lat, df_fortune_500s.Long, df_fortune_500s.Company))

# df_fortune_500s['tup_lat_Long'] = list(zip(data.lat, data.long, data.id))
data['tup_lat_Long'] = list(zip(data.lat, data.long, data.id))

In [37]:
df_Dist_From_Fortune500 = pd.DataFrame(columns = ['id', 'Company', 'Distance'])

distanceConsideredClose = 7
listofValues = []

#looping Through Main mdf
for HLat, HLong, HouseParcel in data['tup_lat_Long']:
    #print(HouseParcel)
    
    #looping through df that contains interesting places 
    for fiveH_Lat, fiveH_Long, fiveH_Company in df_fortune_500s['tup_lat_Long_500']:
        
        #getting the distance between current house and point of interest
        distance = round(hs.haversine((HLat,HLong),(fiveH_Lat,fiveH_Long), unit='mi'),2)
    
        #if point distance < 6 miles insert into a dataframe 
#         if distance < distanceConsideredClose:
        values = [HouseParcel, fiveH_Company, distance]
        listofValues.append(values)

keys = ['id', 'Company', 'Distance']
df_Dist_From_500 = pd.DataFrame(listofValues,columns = keys)


In [38]:
df_Dist_From_500.columns

Index(['id', 'Company', 'Distance'], dtype='object')

In [39]:
df_Dist_From_500["FortuneFiveBin"] = df_Dist_From_500['Distance'].apply(lambda x : "Under10" if x <10 else "Over10" if x <20 else "Over20")
df_Dist_From_500["FortuneFiveBin_2"] = df_Dist_From_500['Distance'].apply(lambda x : "Under7" if x <=7 else "Over7")

df_Dist_From_500.fillna(0, inplace = True)

In [40]:
df_Dist_From_500.isnull().sum()

id                  0
Company             0
Distance            0
FortuneFiveBin      0
FortuneFiveBin_2    0
dtype: int64

In [41]:
df_Dist_From_500.head(5)

,id,Company,Distance,FortuneFiveBin,FortuneFiveBin_2
0,7129300520,Amazon,9,Under10,Over7
1,7129300520,Costco Wholesale,10,Under10,Over7
2,7129300520,Microsoft,11,Over10,Over7
3,7129300520,Starbucks,6,Under10,Under7
4,7129300520,Paccar,10,Under10,Over7


In [42]:
df_Counts_of_Fortune500_FF_3Buckets = df_Dist_From_500[["id", "FortuneFiveBin"]].groupby(["id", "FortuneFiveBin"]).size().reset_index(name='counts')
df_Counts_of_Fortune500_FF_3Buckets
df_Counts_of_Fortune500_FF_2Buckets = df_Dist_From_500[["id", "FortuneFiveBin_2"]].groupby(["id", "FortuneFiveBin_2"]).size().reset_index(name='counts')
df_Counts_of_Fortune500_FF_2Buckets

,id,FortuneFiveBin_2,counts
0,1000102,Over7,22
1,1200019,Over7,9
2,1200019,Under7,2
3,1200021,Over7,9
4,1200021,Under7,2
...,...,...,...
37020,9842300540,Under7,4
37021,9895000040,Over7,10
37022,9895000040,Under7,1
37023,9900000190,Over7,9


In [43]:
df_Counts_of_Fortune500_FF_3Buckets_pivot = pd.pivot_table(df_Counts_of_Fortune500_FF_3Buckets, index='id', columns=['FortuneFiveBin'], values='counts', aggfunc='max')
df_Counts_of_Fortune500_FF_3Buckets_pivot
df_Counts_of_Fortune500_FF_2Buckets_pivot = pd.pivot_table(df_Counts_of_Fortune500_FF_2Buckets, index='id', columns=['FortuneFiveBin_2'], values='counts', aggfunc='max')
df_Counts_of_Fortune500_FF_2Buckets_pivot

FortuneFiveBin_2,Over7,Under7
id,,
1000102,22,nan
1200019,9,2
1200021,9,2
2800031,9,2
3600057,5,6
...,...,...
9842300095,7,4
9842300485,7,4
9842300540,7,4


In [44]:
df_Counts_of_Fortune500_FF_3Buckets_pivot.reset_index()
df_Counts_of_Fortune500_FF_2Buckets_pivot.reset_index()
df_Counts_of_Fortune500_pivot = df_Counts_of_Fortune500_FF_3Buckets_pivot.merge(df_Counts_of_Fortune500_FF_2Buckets_pivot, on="id")

In [45]:
df_Counts_of_Fortune500_pivot = df_Counts_of_Fortune500_pivot.fillna(0)

# Export Files To Excel

In [46]:
df_Counts_of_Fortune500_pivot.to_excel("df_Counts_of_Fortune500_CrossTab.xlsx")